# SMT Translation Model


Statistical Machine Translation is the field of NLP that aims to generate a translation of sentences based on statistics. The goal is that given a sentence $f$ in a foreign language, try to find the target-language sentence $e$ that is most probable to be a translation. This means to find the sentence that maximizes $P(e|f)$. This can be decomposed as:

$$
P(e|f) \propto P(f|e)P(e)
$$

* $P(f|e)$: translation model probability, which measures adequacy, i.e how much of the meaning is preserved in the translation. This model is built based on bilingual data.
* $P(e)$: language model probability, which measures fluency, i.e. how likely the translation is to have been produced by a native speaker of that language. This model is built based on monolingual (target-side) data.

In this note, we are going to describe the steps to create the translation model. More concretely, the phrase table of Moses. The steps are the following:
1. Align the words of the sentence pairs.
2. Extract phrases (subsequences of words) from both languages and pair them.
3. Create a file, the phrase-table, by gathering the phrase pairs and their translation probabilities.


## Word Alignment

In order to know which word in the source side corresponds to which word on the target side, we need the alignment. It is a function that maps words on one side with those in the other.


Having a:
 * sentence $f$ in the source language as a sequence of words $f_1, f_2...f_{|f|}$
 * sentence $e$ in the target language as a sequence of words $e_1, e_2...e_{|e|}$
 * An alignment $a$ that maps words in $e$ with words in $f$.
 
The probability that the sentence $e$ comes from $f$, with an alignment $a$, is computed as a (normalized) product of the translation probabilities of each aligned words:
$$p(e,a|f)=\frac{\epsilon}{(|f|+1)^{|e|}} \prod_{j=1}^{|e|} t(e_j|f_{a(j)})$$

$t(e_j|f_{a(j)})$ represents translation probability of the word $e_j$ to be a translation of its aligned (according to $a$) word  $f_{a(j)}$

In order to compute this, we need to know the translation probabilities. However, from a parallel corpus, we do not know alignments nor translation probabilities. To solve this we follow Expectation-Maximization Algorithm as:
* Expectation step: compute the alignment probabilities based on translation probabilities.
* Maximization step: compute the translation probabilities. based on alignment probabilities.
Keep iterating both steps until converge.

In the following subsection, we describe the EM algorithm more in detail.

### EM Algorithm

Expectation: compute $p(a|e,f)$, the probabilities of every possible alignment (of every sentence pair $e,f$)
  * For each sentence pair $(e,f)$
    * For each possible alignment (this can be seen as a list of aligned word-pairs $(e_j,f_{a(j)})$ ) compute the alignment probability of $a_i$ as:
	  *  Multiply the translation probabilities of the aligned word pairs. $\prod_{j=1}^{|e|} t(e_j|f_{a(j)})$.
	  * Normalize so all the alignment probabilities of $e,f$ sum 1 $\sum_{i=0}^{|f|}  t(e_j|f_{i})$

Maximization: compute $t(e_j|f_k)$, the translation probabilities of every word pair.
* For each word pair $(e_j,f_k)$
  * Collect the counts. Sum the alignment probabilities of those where  $e_j$ and $f_k$ are aligned: $ct(e_j|f_k)=\sum_{a}p(a|e_j,f_k)=p(a_1|e_j,f_k)+p(a_2|e_j,f_k)...$
  * Normalize the counts so they sum 1 (which would be the translation probability): $t(e_j|f_k)=\frac{ct(e_j|f_k)}{ct(*|f_k)}$
  
#### Optional. How was $p(a|e,f)$ (in Expectation step) obtained?

Compute the probability of the alignments:
$$
p(a|e,f)=\frac{p(e,a|f)}{p(e|f)}
$$
The computation of $p(e,a|f)$ has been shown earlier. 

In the case of $p(e|f)$, it is a marginalization out over all the possible alignments:

$$
p(e|f)=\sum_a{p(e,a|f)}=\sum_{a(1)=0}^{|f|} ...\sum_{a(|e|)=0}^{|f|}p(e,a|f)
$$
*(each  $\sum_{a(k)=0}^{|f|}$ indicates all the possibilities that $e_k$ could be aligned to. Next: replace $p(e,a|f)$ with the formula above)*

$$
p(e|f)=\sum_{a(1)=0}^{|f|} ...\sum_{a(|e|)=0}^{|f|}\frac{\epsilon}{(|f|+1)^{|e|}} \prod_{j=1}^{|e|} t(e_j|f_{a(j)})
$$

*(move out $\frac{\epsilon}{(|f|+1)^{|e|}}$. Also exchange $\sum$ and $\prod$. As they are sum of sums, they can be expressed as product and move it inside )*

$$
p(e|f)=\frac{\epsilon}{(|f|+1)^{|e|}} \sum_{a(1)=0}^{|f|} ...\sum_{a(|e|)=0}^{|f|}\prod_{j=1}^{|e|} t(e_j|f_{a(j)})=\frac{\epsilon}{(|f|+1)^{|e|}} \prod_{j=1}^{|e|} \sum_{i=0}^{|f|}  t(e_j|f_{i})
$$


Now that we have the formula for $p(e,a|f)$ and $p(e|f)$ it is possible to compute the probability of the alignment as:
$$
p(a|e,f)=\frac{p(e,a|f)}{p(e|f)}=\frac{\frac{\epsilon}{(|f|+1)^{|e|}} \prod_{j=1}^{|e|} t(e_j|f_{a(j)})}{\frac{\epsilon}{(|f|+1)^{|e|}} \prod_{j=1}^{|e|} \sum_{i=0}^{|f|}  t(e_j|f_{a(j)})}=\\
\prod_{j=1}^{|e|} \frac{t(e_j|f_{a(j)})}{\sum_{i=0}^{|f|}  t(e_j|f_{i})}
$$

This corresponds to a normalized multiplication of the translation probabilities of each $e_j$ in the sentence.

## Phrase extraction

Once the words are aligned, the next step is to extract phrase pairs (i.e. pairs of chunks where the words are aligned to each other). 

From a sentence pair $(f,e)$ subsequence of words $\bar{f}$ and $\bar{e}$ are extracted and they are paired if all the words in one phrase are aligned to those in the counterpart (or NULL, but they cannot be aligned to words that are not included in the phrase).


In the image, we show the example of the alignment of pair of sentences("I have a big car", "yo tengo un coche grande")

![phrase_extraction](images/phrase_extraction.png)

The black squares represent the words that are aligned to each other.

In this pair of sentences, there are several phrases that can be extracted. Not only individual word pairs such ("I","yo"),  ("car","coche"),  ("big","grande") etc. could be extracted, but also subsequence of sentence such as ("I have","yo tengo"), ("big car","coche grande").

However, a phrase pair such as ("a big","un coche grande") are not allowed because the word "coche" is not aligned to any of the words in the English segment. The phrases that can be extracted are those form a square in the next image.


![phrase_extraction_marked](images/phrase_extraction_marked.png)

The green marked areas are made so there is no black square outside the green mark in those rows or columns. Those correspond to the phrase pairs that can be extracted.

## Build phrase-table

Once the phrase pairs are extracted, the next step is to compute the translation probabilities of the phrases. By default, Moses SMT computes four metrics: (i) Inverse translation probability; (ii) Inverse lexical weighting; (iii) Direct phrase translation probability, and (iv) Direct lexical weighting.

The phrase pairs, along with the translation probabilities and other scores (such as the count of occurrences, alignment, etc.) are gathered in Moses into a file called "phrase-table". This file contains rows like the following:


> wenn wir ||| when we ||| 0.2006   0.1772 0.110 0.1551 ||| 0-0 1-1 ||| 648 1177 130 ||| |||



* Phrase in the source side. 
*  Phrase in the target side (paired with the source side phrase)
* Translation model features: 
  * Inverse translation probability: It indicates the probability of a phrase to be the translation of another phrase computed as:
$$\phi(\bar{f}|\bar{e})= \frac{C_{(S,T)}(\bar{f}, \bar{e} )} {\sum\limits_{\bar{f_i}}C_{(S,T)}( \bar{f_i},\bar{e} )}$$
(the count where $\bar{f}$ and $\bar{e}$ occur together divided by how the total $\bar{e}$ is aligned to). The counts of occurrences of the phrases are shown in the last column: "when we" occurs 648 times, "wenn wir" and "when we" occur together 130 times. Therefore the inverse phrase translation probability is $0.2006=130/648$.
  *  Inverse lexical weighting ($lex(f|e)$): This is computed in order to avoid the problem of phrases that do not provide reliable probability estimations (e.g. low-frequency phrases). It measures how well the words in the phrases translate to each other. It is computed the average of alignment of individual words:
$$p_w(\bar{f} |\bar{e},a)=
\prod_{i=1}^{l}\frac{1}{\left \{ j| (i,j)\in a \right \}} \sum_{\forall (i,j)\in a} w(f_i|e_j)$$
  The individual lexical weighting is computed as:
$$w(f_i|e_j)=\frac{C_{(S,T)}(f_i,e_j)}{\sum_{{f}'} C_{(S,T)}({f}',e_j)}$$
the value stored in a file called *lex.e2f* created by Moses. In this file we find the values of lexical weighting for the words in the phrases, in the rows "wenn when 0.2658521" and "wir we 0.6666557". Therefore the inverse lexical weighting is $0.1772=0.2658521 \cdot 0.6666557$. 

  * Direct phrase translation probability ($\phi(e|f)$): The translation probability in the reverse direction. "wenn wir" occurs 1177 times, "wenn wir" and "when we" occur together 130 times. Therefore the direct phrase translation probability is $0.110=130/1177$.
  * Direct lexical weighting ($lex(e|f)$): The lexical weighting in the reverse direction. The individual lexical weighting is stored in a file called *lex.f2e* which contains the rows "when wenn 0.1995174" and "we wir 0.7773823". Therefore the direct lexical weighting is $0.1551=0.1995174 \cdot 0.7773823$.

* Alignments: How words of the source and target side are aligned individually. For example, in the last row, the pair ⟨"wenn wir , when we"⟩ , "0-0" indicate that the 0-th word in the source side word ("wenn") is aligned to the 0-th target-side word ("when").
* Phrase counts:  Three numbers consisting of target phrase counts, source phrase counts, source and target intersection count. For example, in the first example "wenn wir , when we", there are 1177 occurrences of "wenn wir", and 648 occurrences of "when we". The phrases "wenn wir" and "when we" occur together 130 times.


## References

### Word Alignment

https://www.youtube.com/watch?v=mqyMDLu5JPw

https://www.youtube.com/watch?v=nKqyU0bUKTQ)

http://www.statmt.org/book/slides/04-word-based-models.pdf

https://www.csee.umbc.edu/courses/undergraduate/473/f17/content/slides/10-mt.pdf

